In [13]:
from pathlib import Path
import MDAnalysis as mda

from MDAnalysis import Universe
from MDAnalysis.topology.LAMMPSParser import DATAParser
from MDAnalysis.coordinates.PDB import PDBReader, PDBWriter
from MDAnalysis.topology.TOPParser import TOPParser
# from MDAnalysis.topology.PDBParser import PDBParser
# from MDAnalysis.topology.HoomdXMLParser import HoomdXMLParser

### Testing for mbuild pdb completion (filling in bonding info from other formats)

In [16]:
N = 15
mol_name = 'polyethylmethacrylate'
source_dir = Path(f'mbuild_polymers/{mol_name}')
file_stem = f'{mol_name}-N={N}'

pdb_path    = source_dir/f'{file_stem}.pdb'
hoomd_path  = source_dir/f'{file_stem}.hoomdxml'
lammps_path = source_dir/f'{file_stem}.lmp'
gsd_path    = source_dir/f'{file_stem}.gsd'
top_path    = source_dir/f'{file_stem}.top'

# hoomd_parser = HoomdXMLParser(hoomd_path)
# hoomd_parser.parse()

lammps_parser = DATAParser(str(lammps_path))
topo = lammps_parser.parse()
# read = PDBReader(pdb_path)

u = Universe(topo, pdb_path) # use explicit topology but implicit trajectory (traj from pdb)


In [17]:
polymer = u.select_atoms('all')
outpath = Path(f'mbuild_polymers/mda_output/{mol_name}_test.pdb')

with mda.Writer(outpath, multiframe=False) as pdb:
    pdb.write(polymer)

/home/tbernat/miniconda3/envs/polybuild-env/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/home/tbernat/miniconda3/envs/polybuild-env/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'resnames' Using default value of 'UNK'
  warnings.warn("Found no information for attr: '{}'"
/home/tbernat/miniconda3/envs/polybuild-env/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/home/tbernat/miniconda3/envs/polybuild-env/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'chainIDs' Using default value of ''
  warnings.warn("Found no information for attr: '{}'"
/home/tbernat/minicond